In [1]:
from env import github_token, github_username 
import acquire
# import prepare
# import explore
# import model

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

from bs4 import BeautifulSoup as BSoup
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests
import time

import pandas as pd
import numpy as np
import requests

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(13, 10))
plt.rc('font', size=16)
plt.style.use('seaborn-darkgrid')

import warnings
warnings.filterwarnings('ignore')

In [2]:
headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}
p=1
url = f'https://github.com/search?p={p}&q=metaverse&type=Repositories'
url

'https://github.com/search?p=1&q=metaverse&type=Repositories'

In [3]:
#grab the first page
response = requests.get(url, headers=headers)
response.status_code

200

In [4]:
soup=BSoup(response.text)

In [5]:
len(soup.find_all('div', {'class':'f4 text-normal'})) 

10

In [6]:
soup.find_all('div', {'class':'f4 text-normal'})[0:3]

[<div class="f4 text-normal">
 <a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":1,"per_page":10,"query":"metaverse","result_position":1,"click_id":217230234,"result":{"id":217230234,"global_relay_id":"MDEwOlJlcG9zaXRvcnkyMTcyMzAyMzQ=","model_name":"Repository","url":"https://github.com/M3-org/awesome-metaverse"},"originating_url":"https://github.com/search?p=1&amp;q=metaverse&amp;type=Repositories","user_id":null}}' data-hydro-click-hmac="626d41bb1d92644a0f532acb7451d50434eada0ff4d5798446e0e4bab4e32bf5" href="/M3-org/awesome-metaverse">M3-org/awesome-<em>metaverse</em></a>
 </div>,
 <div class="f4 text-normal">
 <a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":1,"per_page":10,"query":"metaverse","result_position":2,"click_id":72335855,"result":{"id":72335855,"global_relay_id":"MDEwOlJlcG9zaXRvcnk3MjMzNTg1NQ==","model_name":"Repository","url":"https://github.com/mvs-org/metave

In [7]:
#seems to grab what we want, so now we just need the href of each one
type(soup.find_all('div', {'class':'f4 text-normal'})[0]) #tag

bs4.element.Tag

In [8]:
soup.find_all('div', {'class':'f4 text-normal'})[0].select_one('a')

<a class="v-align-middle" data-hydro-click='{"event_type":"search_result.click","payload":{"page_number":1,"per_page":10,"query":"metaverse","result_position":1,"click_id":217230234,"result":{"id":217230234,"global_relay_id":"MDEwOlJlcG9zaXRvcnkyMTcyMzAyMzQ=","model_name":"Repository","url":"https://github.com/M3-org/awesome-metaverse"},"originating_url":"https://github.com/search?p=1&amp;q=metaverse&amp;type=Repositories","user_id":null}}' data-hydro-click-hmac="626d41bb1d92644a0f532acb7451d50434eada0ff4d5798446e0e4bab4e32bf5" href="/M3-org/awesome-metaverse">M3-org/awesome-<em>metaverse</em></a>

In [9]:
type(soup.find_all('div', {'class':'f4 text-normal'})[0].select_one('a'))

bs4.element.Tag

In [10]:
soup.find_all('div', {'class':'f4 text-normal'})[0].select_one('a')['href']

'/M3-org/awesome-metaverse'

In [11]:
len(soup.find_all('a',{'class':'v-align-middle'}))

10

In [12]:
#Okay, so the shorter way is:
soup.find_all('a',{'class':'v-align-middle'})[0]['href']

'/M3-org/awesome-metaverse'

In [13]:
soup.find('a',{'class':'next_page'})

<a class="next_page" href="/search?p=2&amp;q=metaverse&amp;type=Repositories" rel="next">Next</a>

In [14]:
soup.find('a',{'class':'next_page'})['href']

'/search?p=2&q=metaverse&type=Repositories'

Test logic for when next page doesn't exist:

In [15]:
#this is the last of many pages
response2 = requests.get('https://github.com/search?p=90&q=metaverse&type=Repositories',headers=headers)
response2.status_code

200

In [16]:
soup2 = BSoup(response2.text)

In [17]:
headers = {"Authorization": f"token {github_token}", "User-Agent": github_username}

def get_repo_links(soup):
    '''
    Given the soup of a repo search page, get a list urls for all the repos in the results
    
    Output: Returns LIST of partial URLs
    Parameters: (R) soup: Beautiful Soup object of the page contents
    '''
    pg_repos = []
    #loop over all our 'a' tags
    for a in soup.find_all('a',{'class':'v-align-middle'}):
        #grab the href and strip first character >> append to repo list
        pg_repos.append(a['href'][1:])
    #return the list 
    return pg_repos
        
def get_metaverse_repos():
    '''
    Gets a list of metaverse repos from Github.  
    Must have your own github token and username in a local env file. 
    Output: Returns LIST of strings - with section of URL in format 'user/repo'
    '''
    #set starting point
    #we know there are multiple pages, so including query parameter 'p'
    p=1
    url = f'https://github.com/search?p={p}&q=metaverse&type=Repositories'
    
    #FIRST PAGE
    #get page info
    response = requests.get(url, headers=headers)
    #SOUP, there it is!
    soup = BSoup(response.text)
    
    #grab links for first page
    repo_list = get_repo_links(soup)
    
    #NEXT PAGE - 
    #set domain
    domain = 'https://github.com'
    #get first next page
    np_tag = soup.find('a',{'class':'next_page'})
    #IF that element exists, go to and repeat!
    while np_tag:
        #create url for next page
        url = domain + np_tag['href']
        
        #get new soup
        response = requests.get(url, headers=headers)
        soup = BSoup(response.text)
        
        #add new links to our list
        repo_list += get_repo_links(soup)
        
        #break loop if we have > X links
        if len(repo_list) >= 5000: break
        
        #grab next page
        np_tag = soup.find('a',{'class':'next_page'})
        time.sleep(7)
        
    return repo_list
    
def create_REPOS():
    filename = 'repos.csv'
    #see if file exists
    if os.path.isfile(filename):
        df = pd.read_csv(filename,header=None)
        return df.iloc[:0].tolist()
    else:
        #get new list of repos
        mylist = get_metaverse_repos()
        #save to CSV
        pd.DataFrame(mylist).to_csv(filename,header=False,index=False)
        return mylist

In [18]:
testing = get_metaverse_repos()
testing[-5:]

['nftport/NFT-Unity3D-Metaverse-Template',
 'LifeScopeLabs/lifescope-xr',
 'GMCyberFoundry/Metaverse',
 'immers-space/immers',
 'Fankouzu/my-metaverse']

In [19]:
len(testing)

70

In [20]:
pd.DataFrame(testing).to_csv('testing.csv',header=False,index=False)

In [21]:
pd.read_csv('testing.csv',header=None)

,0
0,M3-org/awesome-metaverse
1,mvs-org/metaverse
2,webaverse/app
3,shadowcz007/awesome-metaverse
4,vircadia/vircadia
...,...
65,nftport/NFT-Unity3D-Metaverse-Template
66,LifeScopeLabs/lifescope-xr
67,GMCyberFoundry/Metaverse
68,immers-space/immers
